In [1]:
#Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

#ipywidget for demostration purposes
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Create a database connection

In [2]:
conn = sqlite3.connect('renewable_spain.db')
c = conn.cursor()
print("Connection successful")

Connection successful


In [3]:
#print all tables in the database
c.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(c.fetchall())

[('time_stamps',), ('load',), ('energy_dataset',), ('fossil_fuel_generation',)]


Connect to the main table and source data

In [33]:
#read 'energy_dataset' table into a dataframe
df = pd.read_sql_query("SELECT * FROM energy_dataset", conn)

df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17,None,6436,26118,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16,None,5856,24934,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8,None,5454,23515,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2,None,5151,22642,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9,None,4861,21785,20264.0,38.41,56.04


In [38]:
#write a function to perform datetime transformation to the dataframe
def datetime_transform(df):
    df['time'] = pd.to_datetime(df['time'], utc=True)
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df['time'] = pd.to_datetime(df['time'], utc=False)
    df = df.set_index('time', drop=True)
    return df



In [39]:
#specify df to use
plot_df = datetime_transform(df)

In [40]:
#create a function to plot the data, allow user to select a date range, time frequency to plot and column to plot
def plot_data(start_date, end_date, time_freq, column):
    #filter dataframe to only show data between the start and end date
    df_filtered = plot_df.loc[start_date:end_date]
    
    #plot the data
    df_filtered[column].resample(time_freq).mean().plot()
    plt.title(column)
    plt.show()

In [41]:
#Create a dropdown menu to select the column to plot
column = widgets.Dropdown(
    options=plot_df.columns,
    value='total_load_actual',
    description='Column:',
    disabled=False,
)

#Create a dropdown menu to select the time frequency to plot
time_freq = widgets.Dropdown(
    options=['1H', '1D', '1W', '1M'],
    value='1H',
    description='Time Frequency:',
    disabled=False,
)

#Create a date picker to select the start date, restrict the start date to the first date in the dataframe
start_date = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=plot_df.index[0]
)

#Create a date picker to select the end date, restrict the end date to the last date in the dataframe
end_date = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=plot_df.index[-1]
)


In [42]:
#display the widgets and plot the data
interact(plot_data, start_date=start_date, end_date=end_date, time_freq=time_freq, column=column)

interactive(children=(DatePicker(value=Timestamp('2014-12-31 23:00:00'), description='Start Date', step=1), Da…

<function __main__.plot_data(start_date, end_date, time_freq, column)>